In [2]:
import gradio as gr
import pandas as pd
import os
import tempfile
import speech_recognition as sr
import time
import playsound
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv
from transformers import pipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import APIChain
from gtts import gTTS

In [3]:
# Load environment variables.
load_dotenv()

# Get the Gemini API key.
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# Verify that the Gemini API key is set.
if GEMINI_API_KEY is None:
    raise ValueError("The Gemini API key is not set. Please set the GEMINI_API_KEY environment variable.")

# Initialize the Gemini model
#llm = OpenAI(api_key=GEMINI_API_KEY, model="text-davinci-003")

llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model="gemini-1.5-flash", temperature=0.9)

In [4]:
# Text description of API spec.
spec_str = """

Instructions for responding:

Use the Open Library Search API to get up to 4 books related to the person the book is requested for.  
When responding pull any 4 random books from the list do not include any books the users asks to exclude. 
Provide the response to the user with the title and author of the books in natural language. 
Ask the user if they want more information about a book

When a user asks for information about a book by the title:
use Open Library Search API to get the description of the book between 100 and 200 words in length, if there
is no description available tell the user a description isn't available and list 4 similar books.

When a user asks for information about the author:
use Open Library Search API to get information about the author.
Respond to the user by telling them about the author and 
use Open Library Search API to provide 2 more books written by that author.

Open Library provides an experimental API to search.

WARNING: This API is under active development and may change in future.

Overview & Features
The Open Library Search API is one of the most convenient and complete ways to retrieve book data on Open Library. The API:

Is able to return data for multiple books in a single request/response
Returns both Work level information about the book, as well as Edition level information (such as)
Author IDs are returned which you can use to fetch the author's image, if available
Options are available to return Book Availability along with the response.
Powerful sorting options are available, such as star ratings, publication date, and number of editions.
Endpoint
The endpoint for this API is:
https://openlibrary.org/search.json

Examples
The URL format for API is simple. Take the search URL and add .json to the end. Eg:

https://openlibrary.org/search.json?q=the+lord+of+the+rings
https://openlibrary.org/search.json?title=the+lord+of+the+rings
https://openlibrary.org/search.json?author=tolkien&sort=new
https://openlibrary.org/search.json?q=the+lord+of+the+rings&page=2
https://openlibrary.org/search/authors.json?q=twain
Using Thing IDs to get Images
You can use the olid (Open Library ID) for authors and books to fetch covers by olid, e.g.:
https://covers.openlibrary.org/a/olid/OL23919A-M.jpg

URL Parameters
Parameter	Description
q	The solr query. See Search HowTo for sample queries
fields	The fields to get back from solr. Use the special value * to get all fields (although be prepared for a very large response!).
To fetch availability data from archive.org, add the special value, availability. Example: /search.json?q=harry%20potter&fields=*,availability&limit=1. This will fetch the availability data of the first item in the `ia` field.
sort	You can sort the results by various facets such as new, old, random, or key (which sorts as a string, not as the number stored in the string). For a complete list of sorts facets look here (this link goes to a specific commit, be sure to look at the latest one for changes). The default is to sort by relevance.
lang	The users language as a two letter (ISO 639-1) language code. This influences but doesn't exclude search results. For example setting this to fr will prefer/display the French edition of a given work, but will still match works that don't have French editions. Adding language:fre on the other hand to the search query will exclude results that don't have a French edition.
offset / limit	Use for pagination.
page / limit	Use for pagination, with limit corresponding to the page size. Note page starts at 1.
Include no other text, only the API call URL. Don't use newlines."""

In [5]:
# Define chatbot function.
def run(msg, history):
    query = {"question": msg}
    chain = APIChain.from_llm_and_api_docs(llm, spec_str, limit_to_domains=["https://openlibrary.org"])

    try:
        # Get the text response from APIChain.
        response = chain.invoke(query)
        text_response = response ["output"] 

        # Convert the text response to speech.
        tts = gTTS(text_response, lang='en')
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tts.save(temp_file.name)
        return text_response, temp_file.name
        #return error_message, temp_file.name
    
    except Exception as e:
        error_message = f"Sorry, an error occurred: {e}. Please try again."
        tts = gTTS(error_message, lang='en')
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tts.save(temp_file.name)
        return error_message, temp_file.name
    
# Create a Gradio interface.
app = gr.Interface(
    fn=run,
    inputs="text",
    outputs=[
        "text",
        "audio"         
    ],
    title="Elf Chatbot",
    description="This Chatbot can help you find a book for someone.",
)

# Launch the Gradio interface.
app.launch(share=True)                            

c:\Users\azamora\AppData\Local\anaconda3\Lib\site-packages\gradio\utils.py:1003: UserWarning: Expected 2 arguments for function <function run at 0x00000184A7D83880>, received 1.
  warnings.warn(
c:\Users\azamora\AppData\Local\anaconda3\Lib\site-packages\gradio\utils.py:1007: UserWarning: Expected at least 2 arguments for function <function run at 0x00000184A7D83880>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


c:\Users\azamora\AppData\Local\anaconda3\Lib\site-packages\gradio\helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
c:\Users\azamora\AppData\Local\anaconda3\Lib\site-packages\gradio\helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
c:\Users\azamora\AppData\Local\anaconda3\Lib\site-packages\gradio\helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
c:\Users\azamora\AppData\Local\anaconda3\Lib\site-packages\gradio\helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
